In [1]:
# Dependencies and Setup

import json
import pandas as pd
import requests
import numpy as py
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine, inspect, func, distinct
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3
from sqlite3 import Error

In [2]:
#Bring in data from internet as object then convert ot Json and count number of games

# REQUEST
schedule_response_obj=requests.get("http://data.nba.net/prod/v1/2018/schedule.json")
schedule_response=schedule_response_obj.json()

# Count the number of Games for the season inschedule
games_response_count=schedule_response["league"]["standard"]
seasongame_number=len(games_response_count)
# seasongame_number

In [3]:
###   Collect Elements for URL, append List, create DataFrame, DF_exclude games not played  ###

# Bring in Game Id and GameData ("startDataEastern") elements from NBA Schedule later used to create box_score URL

# Get elements
NBA_Schedule=[]
for item in schedule_response["league"]["standard"]:
    item_dict={}
    item_dict["gameId"]=item["gameId"]
    item_dict["startDateEastern"]=item["startDateEastern"]
    item_dict["statusNum"]=item["statusNum"]

# Append List
    NBA_Schedule.append({"gameid":item_dict["gameId"],
                         "startdateeastern":item_dict["startDateEastern"],
                          "statusnum":item_dict["statusNum"]})


# Convert NBA_Schedule=[] list to DataFrame and modify Datatypes
NBA_Schedule_DF_initial=pd.DataFrame(NBA_Schedule)
NBA_Schedule_DF_initial["gameid"]=NBA_Schedule_DF_initial["gameid"].astype(int)
NBA_Schedule_DF_initial["startdateeastern"]=NBA_Schedule_DF_initial["startdateeastern"].astype(int)
NBA_Schedule_DF_initial["statusnum"]=NBA_Schedule_DF_initial["statusnum"].astype(int)

# Exclude Games not Yet Played
# OPEN: Exclude Pre-season Games
NBA_Schedule_PlayedGM_DF=NBA_Schedule_DF_initial.loc[NBA_Schedule_DF_initial["statusnum"]==3,:]

In [4]:
## Count Games Played, Generate URL for each Game Played, and add to DF: NBA_Schedule_PlayedGM_DF

# count the number of gamesId records collected (note: some are pre-season and some haven't been played)
playedgame_count=len(NBA_Schedule_PlayedGM_DF["gameid"])

In [5]:
# Generate Box_Score_URL and append to List Box_Score_URL_List
Box_score_URL_List=[]

# # Loop to generate Box Score URL using Gameid and GameData ("startDataEastern")

for game_box in range (0,playedgame_count):
    Box_score_URL=F"http://data.nba.net/prod/v1/{NBA_Schedule_PlayedGM_DF['startdateeastern'][game_box]}/00{NBA_Schedule_PlayedGM_DF['gameid'][game_box]}_boxscore.json"
#     Append list
    Box_score_URL_List.append(Box_score_URL)
    
NBA_Schedule_PlayedGM_DF["boxscoreURL"]=Box_score_URL_List
NBA_Schedule_PlayedGM_DF.dtypes

C:\Users\JC\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


gameid               int32
startdateeastern     int32
statusnum            int32
boxscoreURL         object
dtype: object

In [20]:
# Box Score Request and request Object for i in range (0,playedgame_count) THEN
# Loop through ALL Box Scores: Extract vteam and hteam basic box score data EXCLUDING: INDIVIDUAL PLAYER STATS 

Team_boxscores_list=[]
for i in range (0,playedgame_count):
    box_score_obj=requests.get(NBA_Schedule_PlayedGM_DF["boxscoreURL"][i])
    box_score_response=box_score_obj.json()
    vteam_dict={}
    vteam_dict["gameId"]=box_score_response["basicGameData"]["gameId"]
    vteam_dict["teamId"]=box_score_response["basicGameData"]["vTeam"]["teamId"]
    vteam_dict["fastBreakPoints"]=box_score_response["stats"]["vTeam"]["fastBreakPoints"]
    vteam_dict["pointsInPaint"]=box_score_response["stats"]["vTeam"]["pointsInPaint"]
    vteam_dict["secondChancePoints"]=box_score_response["stats"]["vTeam"]["secondChancePoints"]
    vteam_dict["pointsOffTurnovers"]=box_score_response["stats"]["vTeam"]["pointsOffTurnovers"]
    vteam_dict["points"]=box_score_response["stats"]["vTeam"]["totals"]["points"]
    vteam_dict["fgm"]=box_score_response["stats"]["vTeam"]["totals"]["fgm"]
    vteam_dict["fga"]=box_score_response["stats"]["vTeam"]["totals"]["fga"]
    vteam_dict["fgp"]=box_score_response["stats"]["vTeam"]["totals"]["fgp"]
    vteam_dict["ftm"]=box_score_response["stats"]["vTeam"]["totals"]["ftm"]
    vteam_dict["fta"]=box_score_response["stats"]["vTeam"]["totals"]["fta"]
    vteam_dict["ftp"]=box_score_response["stats"]["vTeam"]["totals"]["ftp"]
    vteam_dict["tpm"]=box_score_response["stats"]["vTeam"]["totals"]["tpm"]
    vteam_dict["tpa"]=box_score_response["stats"]["vTeam"]["totals"]["tpa"]
    vteam_dict["tpp"]=box_score_response["stats"]["vTeam"]["totals"]["tpp"]
    vteam_dict["offReb"]=box_score_response["stats"]["vTeam"]["totals"]["offReb"]
    vteam_dict["defReb"]=box_score_response["stats"]["vTeam"]["totals"]["defReb"]
    vteam_dict["totReb"]=box_score_response["stats"]["vTeam"]["totals"]["totReb"]
    vteam_dict["assists"]=box_score_response["stats"]["vTeam"]["totals"]["assists"]
    vteam_dict["pFouls"]=box_score_response["stats"]["vTeam"]["totals"]["pFouls"]
    vteam_dict["steals"]=box_score_response["stats"]["vTeam"]["totals"]["steals"]
    vteam_dict["turnovers"]=box_score_response["stats"]["vTeam"]["totals"]["turnovers"]
    vteam_dict["blocks"]=box_score_response["stats"]["vTeam"]["totals"]["blocks"]
    vteam_dict["plusMinus"]=box_score_response["stats"]["vTeam"]["totals"]["plusMinus"]

    Team_boxscores_list.append({"gameid":vteam_dict["gameId"],
                                "teamid":vteam_dict["teamId"],
                                "fastbreakpoints":vteam_dict["fastBreakPoints"],
                                "pointsinpaint":vteam_dict["pointsInPaint"],
                                "secondchancepoints":vteam_dict["secondChancePoints"],
                                "pointsoffturnovers":vteam_dict["pointsOffTurnovers"],
                                "points":vteam_dict["points"],
                                "fgm":vteam_dict["fgm"],
                                "fga":vteam_dict["fga"],
                                "fgp":vteam_dict["fgp"],
                                "ftm":vteam_dict["ftm"],
                                "fta":vteam_dict["fta"],
                                "ftp":vteam_dict["ftp"],
                                "tpm":vteam_dict["tpm"],
                                "tpa":vteam_dict["tpa"],
                                "tpp":vteam_dict["tpp"],
                                "offreb":vteam_dict["offReb"],
                                "defreb":vteam_dict["defReb"],
                                "totreb":vteam_dict["totReb"],
                                "assists":vteam_dict["assists"],
                                "pfouls":vteam_dict["pFouls"],
                                "steals":vteam_dict["steals"],
                                "turnovers":vteam_dict["turnovers"],
                                "blocks":vteam_dict["blocks"],
                                "plusminus":vteam_dict["plusMinus"]})

    hteam_dict={}
    hteam_dict["gameId"]=box_score_response["basicGameData"]["gameId"]
    hteam_dict["teamId"]=box_score_response["basicGameData"]["hTeam"]["teamId"]
    hteam_dict["fastBreakPoints"]=box_score_response["stats"]["hTeam"]["fastBreakPoints"]
    hteam_dict["pointsInPaint"]=box_score_response["stats"]["hTeam"]["pointsInPaint"]
    hteam_dict["secondChancePoints"]=box_score_response["stats"]["hTeam"]["secondChancePoints"]
    hteam_dict["pointsOffTurnovers"]=box_score_response["stats"]["hTeam"]["pointsOffTurnovers"]
    hteam_dict["points"]=box_score_response["stats"]["hTeam"]["totals"]["points"]
    hteam_dict["fgm"]=box_score_response["stats"]["hTeam"]["totals"]["fgm"]
    hteam_dict["fga"]=box_score_response["stats"]["hTeam"]["totals"]["fga"]
    hteam_dict["fgp"]=box_score_response["stats"]["hTeam"]["totals"]["fgp"]
    hteam_dict["ftm"]=box_score_response["stats"]["hTeam"]["totals"]["ftm"]
    hteam_dict["fta"]=box_score_response["stats"]["hTeam"]["totals"]["fta"]
    hteam_dict["ftp"]=box_score_response["stats"]["hTeam"]["totals"]["ftp"]
    hteam_dict["tpm"]=box_score_response["stats"]["hTeam"]["totals"]["tpm"]
    hteam_dict["tpa"]=box_score_response["stats"]["hTeam"]["totals"]["tpa"]
    hteam_dict["tpp"]=box_score_response["stats"]["hTeam"]["totals"]["tpp"]
    hteam_dict["offReb"]=box_score_response["stats"]["hTeam"]["totals"]["offReb"]
    hteam_dict["defReb"]=box_score_response["stats"]["hTeam"]["totals"]["defReb"]
    hteam_dict["totReb"]=box_score_response["stats"]["hTeam"]["totals"]["totReb"]
    hteam_dict["assists"]=box_score_response["stats"]["hTeam"]["totals"]["assists"]
    hteam_dict["pFouls"]=box_score_response["stats"]["hTeam"]["totals"]["pFouls"]
    hteam_dict["steals"]=box_score_response["stats"]["hTeam"]["totals"]["steals"]
    hteam_dict["turnovers"]=box_score_response["stats"]["hTeam"]["totals"]["turnovers"]
    hteam_dict["blocks"]=box_score_response["stats"]["hTeam"]["totals"]["blocks"]
    hteam_dict["plusMinus"]=box_score_response["stats"]["hTeam"]["totals"]["plusMinus"]

    Team_boxscores_list.append({"gameid":hteam_dict["gameId"],
                                "teamid":hteam_dict["teamId"],
                                "fastbreakpoints":hteam_dict["fastBreakPoints"],
                                "pointsinpaint":hteam_dict["pointsInPaint"],
                                "secondchancepoints":hteam_dict["secondChancePoints"],
                                "pointsoffturnovers":hteam_dict["pointsOffTurnovers"],
                                "points":hteam_dict["points"],
                                "fgm":hteam_dict["fgm"],
                                "fga":hteam_dict["fga"],
                                "fgp":hteam_dict["fgp"],
                                "ftm":hteam_dict["ftm"],
                                "fta":hteam_dict["fta"],
                                "ftp":hteam_dict["ftp"],
                                "tpm":hteam_dict["tpm"],
                                "tpa":hteam_dict["tpa"],
                                "tpp":hteam_dict["tpp"],
                                "offreb":hteam_dict["offReb"],
                                "defreb":hteam_dict["defReb"],
                                "totreb":hteam_dict["totReb"],
                                "assists":hteam_dict["assists"],
                                "pfouls":hteam_dict["pFouls"],
                                "steals":hteam_dict["steals"],
                                "turnovers":hteam_dict["turnovers"],
                                "blocks":hteam_dict["blocks"],
                                "plusminus":hteam_dict["plusMinus"]})

In [21]:
Team_boxscores_df=pd.DataFrame(Team_boxscores_list)
Team_boxscores_df["gameid"]=Team_boxscores_df["gameid"].astype(int)
Team_boxscores_df["teamid"]=Team_boxscores_df["teamid"].astype(int)
Team_boxscores_df["fastbreakpoints"]=Team_boxscores_df["fastbreakpoints"].astype(float)
Team_boxscores_df["pointsinpaint"]=Team_boxscores_df["pointsinpaint"].astype(float)
Team_boxscores_df["secondchancepoints"]=Team_boxscores_df["secondchancepoints"].astype(float)
Team_boxscores_df["pointsoffturnovers"]=Team_boxscores_df["pointsoffturnovers"].astype(float)
Team_boxscores_df["points"]=Team_boxscores_df["points"].astype(float)
Team_boxscores_df["fgm"]=Team_boxscores_df["fgm"].astype(float)
Team_boxscores_df["fga"]=Team_boxscores_df["fga"].astype(float)
Team_boxscores_df["fgp"]=Team_boxscores_df["fgp"].astype(float)
Team_boxscores_df["ftm"]=Team_boxscores_df["ftm"].astype(float)
Team_boxscores_df["fta"]=Team_boxscores_df["fta"].astype(float)
Team_boxscores_df["ftp"]=Team_boxscores_df["ftp"].astype(float)
Team_boxscores_df["tpm"]=Team_boxscores_df["tpm"].astype(float)
Team_boxscores_df["tpa"]=Team_boxscores_df["tpa"].astype(float)
Team_boxscores_df["tpp"]=Team_boxscores_df["tpp"].astype(float)
Team_boxscores_df["offreb"]=Team_boxscores_df["offreb"].astype(float)
Team_boxscores_df["defreb"]=Team_boxscores_df["defreb"].astype(float)
Team_boxscores_df["totreb"]=Team_boxscores_df["totreb"].astype(float)
Team_boxscores_df["assists"]=Team_boxscores_df["assists"].astype(float)
Team_boxscores_df["pfouls"]=Team_boxscores_df["pfouls"].astype(float)
Team_boxscores_df["steals"]=Team_boxscores_df["steals"].astype(float)
Team_boxscores_df["turnovers"]=Team_boxscores_df["turnovers"].astype(float)
Team_boxscores_df["blocks"]=Team_boxscores_df["blocks"].astype(float)
Team_boxscores_df["plusminus"]=Team_boxscores_df["plusminus"].astype(float)
Team_boxscores_df.dtypes

assists               float64
blocks                float64
defreb                float64
fastbreakpoints       float64
fga                   float64
fgm                   float64
fgp                   float64
fta                   float64
ftm                   float64
ftp                   float64
gameid                  int32
offreb                float64
pfouls                float64
plusminus             float64
points                float64
pointsinpaint         float64
pointsoffturnovers    float64
secondchancepoints    float64
steals                float64
teamid                  int32
totreb                float64
tpa                   float64
tpm                   float64
tpp                   float64
turnovers             float64
dtype: object

In [14]:
engine = create_engine("sqlite:///db/NBA_Data.sqlite")
conn = engine.connect()

engine.table_names()

['All_Teams',
 'NBA_Fantasy_Draft',
 'Team_Locations',
 'Team_Schedule',
 'Team_Stats',
 'season_2016_2017',
 'season_2017_2018']

In [23]:
Team_boxscores_df.set_index("teamid", inplace=True)

#Load Team_Stats table
Team_boxscores_df.to_sql('Team_Stats',conn, if_exists='replace', index=True)